In [32]:
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

def get_number(s):
    try:
        return float(s.replace(",","."))
    except ValueError:
        return None

#Open URL and create RoboBrowser Instance
browser = RoboBrowser(parser='html.parser', history=False)
dates = pd.date_range('2016-03-27', periods=2, freq='W').format('YYYY-MM-dd')[1:] # weekly dates
print(dates)
results = pd.DataFrame()

for d in dates:
    
    url='https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/' + d + '/DE'
    print(url)
    browser.open(url)  
    soup = bs(str(browser.parsed), 'html.parser') # get html
    germanData = soup.find(id="tab_de")
    weekData = germanData.find("table", class_="hours") # select german hourly table (last on tob DE/AT)
    #print(soup.prettify())
    dates = pd.date_range(pd.to_datetime(d) - pd.to_timedelta(6, 'D'), periods=7, freq="D") # create dates for week
    hours = [h.get_text().strip() for h in weekData.select(".no-border .title")] # get table column names
    priceRows = weekData.findAll("tr", class_="no-border") # get rows with €/Mwh
    prices = np.zeros((24, 7))
    for i in range(24):
        prices[i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")[2:]]
    prices = prices.transpose()
    df = pd.DataFrame(prices)
    df.index = dates
    df.columns = hours
    # cut off last dates of 2015
    if d == "2016-01-03":
        df = df[4:]
    #print(dates)
    #print(hours)
    #print(prices)
 
    #print(df)
    if results.empty:
        results = df
    else:
        results = results.append(df)
print(results)


['2016-03-27', '2016-04-03']
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2016-03-27/DE
https://www.epexspot.com/de/marktdaten/dayaheadauktion/auction-table/2016-04-03/DE
            00 - 01  01 - 02  02 - 03  03 - 04  04 - 05  05 - 06  06 - 07  \
2016-03-21    20.82    19.90    19.79    19.90    20.40    23.18    32.01   
2016-03-22    20.28    20.08    20.05    20.47    21.00    22.01    24.07   
2016-03-23    21.30    20.51    20.26    21.06    21.38    21.38    25.88   
2016-03-24    22.08    21.50    21.11    21.38    21.75    22.97    31.49   
2016-03-25    23.92    21.07    21.00    20.00    20.01    20.30    20.26   
2016-03-26    28.07    24.08    21.11    20.00    20.00    20.01    20.01   
2016-03-27     7.33     3.04      NaN     6.10     4.02     4.02     0.91   
2016-03-28    12.88     8.28     7.43     6.50     5.27     6.28     5.86   
2016-03-29    11.51     9.14    11.95     9.23     8.30    13.49    25.35   
2016-03-30    24.90    22.00    20.